# 使用 MCP 构建代理：AI 与外部资源的无缝集成

## 介绍

模型上下文协议 (MCP) 是一种开放协议，旨在标准化应用程序如何为大型语言模型 (LLM) 提供上下文。可以将 MCP 想象成 AI 应用程序的 USB-C 端口 - 就像 USB-C 提供了一种连接设备和各种外围设备的标准化方式一样，MCP 提供了一种连接 AI 模型和不同数据源及工具的标准化方式。

本教程将指导您在 AI 代理应用程序中实现 MCP，演示它如何通过提供对外部资源、工具和数据源的无缝访问来增强代理的能力。

## 为什么 MCP 对代理很重要

将 AI 模型与外部资源连接的传统方法通常涉及为每个数据源或工具进行自定义集成。这导致：

- **集成复杂性**：每个新数据源都需要独特的实现
- **可扩展性问题**：添加新工具变得越来越困难
- **维护开销**：更新一个集成可能会破坏其他集成

MCP 通过提供标准化协议解决了这些挑战，该协议支持：

- **统一访问**：用于多个数据源和工具的单一接口
- **即插即用扩展**：轻松添加新功能
- **有状态通信**：AI 与资源之间的实时双向通信
- **动态发现**：AI 可以即时发现并使用新工具

以下是重点介绍官方 MCP 服务器示例的简明段落：

## 官方 MCP 服务器示例

MCP 社区维护了一系列参考服务器实现，展示了最佳实践并演示了各种集成模式。这些官方示例可在 [MCP Servers](https://github.com/modelcontextprotocol/servers/tree/main/src) 获得，为希望创建自己的 MCP 服务器的开发人员提供了宝贵的起点。

## 我们将构建什么

在本教程中，我们将实现：

1. **构建您的 MCP 服务器并使用它**：构建一个带有自定义工具的 MCP 服务器并连接到 Claude Desktop
2. **支持自定义工具的代理**：创建一个可以通过 MCP 使用外部工具的自定义代理

在本教程结束时，您将了解 MCP 如何通过让 AI 代理访问更广泛的数字生态系统来增强其能力，使其功能更强大、具有上下文感知能力且更有用。

让我们从了解 MCP 架构和设置环境开始！

## MCP 架构概述

![MCP 架构](../images/mcp_architecture.png)

MCP 遵循包含三个主要组件的客户端-服务器架构：

- **主机**：需要访问外部资源的 AI 应用程序（如 Claude Desktop、Cursor 或自定义代理）
- **客户端**：维护与服务器连接的连接器
- **服务器**：通过 MCP 协议公开能力（数据、工具、提示）的轻量级程序
- **数据源**：MCP 服务器可以访问的本地（文件、数据库）和远程服务（API）

MCP 内的通信使用基于 WebSocket 连接的 JSON-RPC 2.0，确保组件之间的实时双向通信。

## 体验 MCP：在构建之前尝试

虽然本教程侧重于构建您自己的 MCP 服务器并将其与 AI 代理集成，但您可能希望在深入开发之前快速体验 MCP 在实践中的工作原理。

官方 MCP 文档为希望在 Claude Desktop 或其他兼容 AI 应用程序中尝试现有 MCP 服务器的用户提供了出色的快速入门指南。这让您无需编写任何代码即可亲身体验 MCP 启用的功能。

**👉 亲自尝试：** [MCP 用户快速入门指南](https://modelcontextprotocol.io/quickstart/user)

通过探索快速入门指南，您将对我们在本教程中构建的内容获得实际的见解。当您准备好了解内部工作原理并创建自己的实现时，请继续下面的分步开发过程。

现在，让我们开始构建我们自己的 MCP 服务器和客户端！

## 构建您的 MCP 服务器

既然我们要了解 MCP 的基础知识，那就让我们构建第一个 MCP 服务器吧！在本节中，我们将使用 CoinGecko API 创建一个加密货币价格查询服务。我们的服务器将提供允许 AI 检查加密货币当前价格或市场数据的工具。

### 设置我们的环境

在深入实施之前，让我们安装必要的包并设置我们的环境。

> **注意：** 有关安装步骤，请打开终端窗口。这些命令应在常规终端中运行，而不是在 Jupyter 笔记本单元格中。

#### 步骤 1：安装 uv 包管理器

```bash
# 在终端中运行此命令，而不是在 Jupyter 中
curl -LsSf https://astral.sh/uv/install.sh | sh
```

#### 步骤 2：设置项目

```bash
# 创建并导航到项目目录
mkdir mcp-crypto-server
cd mcp-crypto-server
uv init

# 创建并激活虚拟环境
uv venv
source .venv/bin/activate  # 在 Windows 上: .venv\Scripts\activate

# 安装依赖项
uv add "mcp[cli]" httpx
```

### 运行 MCP 服务器

在我们设置好环境后，我们可以开始构建我们的工具。

请查看 [mcp_server.py](scripts/mcp_server.py) 以了解如何构建工具。


现在我们可以通过在终端中运行以下命令来启动服务器：
```bash
# 从 scripts 文件夹复制服务器文件
cp ../scripts/mcp_server.py .

# 启动 MCP 服务器 
uv run mcp_server.py
```

### 与 Claude Desktop 集成
如果您尚未下载 Claude Desktop，请查看 [此页面](https://claude.ai/download)。

要将您的 MCP 服务器连接到 Claude Desktop：

#### 步骤 1：找到 uv 命令的绝对路径：
```bash
which uv
```
复制输出（例如 /user/local/bin/uv 或类似内容）

#### 步骤 2：创建或编辑 Claude Desktop 配置文件：

- 在 macOS 上: ~/Library/Application Support/Claude/claude_desktop_config.json
- 在 Windows 上: %APPDATA%\Claude\claude_desktop_config.json
- 在 Linux 上: ~/.config/Claude/claude_desktop_config.json

您可以查看 [此页面](https://modelcontextprotocol.io/quickstart/user#2-add-the-filesystem-mcp-server) 以了解如何创建配置文件。

#### 步骤 3：添加您的 MCP 服务器配置：

```json
{
    "mcpServers": {
        "crypto-price-tracker": {
            "command": "/ABSOLUTE/PATH/TO/uv",
            "args": [
                "--directory",
                "/ABSOLUTE/PATH/TO/GenAI_Agents/all_agents_tutorials/mcp-crypto-server",
                "run",
                "mcp_server.py"
            ]
        }
    }
}
```
将 `/ABSOLUTE/PATH/TO/uv` 替换为您从 `which uv` 命令获得的路径，并将 `/ABSOLUTE/PATH/TO/GenAI_Agents` 替换为您的存储库的绝对路径。


#### 步骤 4：重新启动 Claude Desktop 以使更改生效。

您应该在聊天框中看到此锤子图标。

![连接了 MCP 的 Claude Desktop](../images/Claude_Desktop_with_MCP.png)

#### 步骤 5：尝试询问比特币的价格

输入“What is the current price of Bitcoin ?”，您将得到如下回复：

![使用 MCP 跟踪比特币价格](../images/track_bitcoin_price_with_mcp.png)


恭喜！您已成功应用您的 MCP 服务器和工具。现在，您可以尝试将自己的工具添加到 [mcp_server.py](/mcp-crypto-server/mcp_server.py)。这是一个例子：

```python
@mcp.tool()
async def get_crypto_market_info(crypto_ids: str, currency: str = "usd") -> str:
    """
    获取一种或多种加密货币的市场信息。
    
    参数：
    - crypto_ids：逗号分隔的加密货币 ID 列表（例如 'bitcoin,ethereum'）
    - currency：显示数值的货币（默认：'usd'）
    
    返回：
    - 包括价格、市值、交易量和价格变化在内的市场信息
    """
    # 构建 API URL
    url = f"{COINGECKO_BASE_URL}/coins/markets"
    
    # 设置查询参数
    params = {
        "vs_currency": currency,  # 显示数值的货币
        "ids": crypto_ids,        # 逗号分隔的加密货币 ID
        "order": "market_cap_desc", # 按市值排序
        "page": 1,                # 页码
        "sparkline": "false"      # 排除迷你图数据
    }
    
    try:
        # 进行 API 调用
        async with httpx.AsyncClient() as client:
            response = await client.get(url, params=params)
            response.raise_for_status()
            
            # 解析响应
            data = response.json()
            
            # 检查我们是否获得了数据
            if not data:
                return f"No data found for cryptocurrencies: '{crypto_ids}'. Please check the IDs and try again."
            
            # 格式化结果
            result = ""
            for crypto in data:
                name = crypto.get('name', 'Unknown')
                symbol = crypto.get('symbol', '???').upper()
                price = crypto.get('current_price', 'Unknown')
                market_cap = crypto.get('market_cap', 'Unknown')
                volume = crypto.get('total_volume', 'Unknown')
                price_change = crypto.get('price_change_percentage_24h', 'Unknown')
                
                result += f"{name} ({symbol}):\n"
                result += f"Current price: {price} {currency.upper()}\n"
                result += f"Market cap: {market_cap} {currency.upper()}\n"
                result += f"24h trading volume: {volume} {currency.upper()}\n"
                result += f"24h price change: {price_change}%\n\n"
            
            return result
            
    except Exception as e:
        return f"Error fetching market data: {str(e)}"
```

使用 `uv run mcp_server.py` 重新运行您的 mcp 服务器，重新启动 Claude Desktop，然后输入“What's the market data for Dogecoin and Solana?”。您将获得如下响应：

![使用 MCP 跟踪加密货币市场数据](../images/track_crypto_market_data_with_mcp.png)

## 通过 MCP 执行工具的自定义代理

在我们构建自己的 MCP 后，让我们尝试自己构建 MCP 主机和客户端。

### 理解架构

在本节中，我们将构建自己的 MCP 主机和客户端。与之前连接到 Claude Desktop 的方法不同，我们现在将创建我们自己的代理，它可以：
1. 充当 MCP 主机
2. 从我们的 MCP 服务器发现可用工具
3. 根据用户查询了解何时使用哪种工具
4. 使用适当的参数执行工具
5. 处理工具结果以提供有用的响应

此架构遵循现代 AI 系统中常见的模式：
- **发现阶段**：我们的自定义主机发现哪些工具可用
- **规划阶段**：代理根据用户的查询决定使用哪种工具
- **执行阶段**：我们的客户端连接到服务器并执行选定的工具
- **解释阶段**：代理解释自然语言中的结果

这是一个简单的工作流图：

![使用 MCP 跟踪加密货币市场数据](../images/customized_mcp_host.png)

运行代码前的重要提示：
⚠️ 别忘了先启动您的 MCP 服务器！⚠️
在运行以下教程中的代理代码之前，请确保您的 MCP 服务器已启动并正在运行。否则，您的代理将没有任何工具可供发现或执行。

让我们从设置环境和导入必要的库开始：

In [1]:
! pip install mcp anthropic

我们需要两个主要库：
- **MCP**：处理与我们的 MCP 服务器的客户端-服务器通信，允许我们构建主机和客户端组件
- **Anthropic**：与 Claude 交互，它将为我们的代理提供推理能力

现在，让我们为我们的代理设置必要的导入和配置：

In [ ]:
# Import necessary libraries
import os
import json
from typing import List, Dict, Any

# MCP libraries for connecting to server
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Anthropic API for Claude
from anthropic import Anthropic

# Set up Anthropic API key (using the one you provided)
os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key_here"

# Initialize the Anthropic client
client = Anthropic()

# Path to your MCP server
mcp_server_path = "absolute/path/to/your/running/mcp/server"
print("Setup complete!")

Setup complete!


我们正在使用 MCP 的 `stdio_client` 接口，该接口允许我们连接到作为单独进程运行并通过标准输入/输出进行通信的 MCP 服务器。这是本地开发的一种简单而稳健的方法。通过实现 MCP 协议的双方（主机和客户端），我们可以完全控制我们的代理如何与 MCP 工具交互。

### 工具发现：构建我们的 MCP 主机

构建自定义 MCP 实现的第一步是创建一个主机，该主机可以发现我们的 MCP 服务器提供了哪些工具。我们的主机将充当用户、AI 和可用工具之间的中介 - 类似于 Claude Desktop 的功能，但完全在我们的控制之下。

让我们实现一个连接到我们的 MCP 服务器并发现其工具的函数：

In [2]:
async def discover_tools():
    """
    Connect to the MCP server and discover available tools.
    Returns information about the available tools.
    """
    # ANSI color codes for better log visibility
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    RESET = "\033[0m"
    SEP = "=" * 40
    
    # Create server parameters for connecting to your MCP server through stdio
    server_params = StdioServerParameters(
        command="python",  # Command to run the server
        args=[mcp_server_path],  # Path to your MCP server script
    )
    
    print(f"{BLUE}{SEP}\n🔍 DISCOVERY PHASE: Connecting to MCP server...{RESET}")
    
    # Connect to the server via stdio
    async with stdio_client(server_params) as (read, write):
        # Create a client session
        async with ClientSession(read, write) as session:
            # Initialize the connection
            print(f"{BLUE}📡 Initializing MCP connection...{RESET}")
            await session.initialize()
            
            # List the available tools
            print(f"{BLUE}🔎 Discovering available tools...{RESET}")
            tools = await session.list_tools()
            
            # Format the tools information for easier viewing
            tool_info = []
            for tool_type, tool_list in tools:
                if tool_type == "tools":
                    for tool in tool_list:
                        tool_info.append({
                            "name": tool.name,
                            "description": tool.description,
                            "schema": tool.inputSchema
                        })
            
            print(f"{GREEN}✅ Successfully discovered {len(tool_info)} tools{RESET}")
            print(f"{SEP}")
            return tool_info

print("Tool discovery function defined")

Tool discovery function defined


此函数充当我们主机的发现组件：

1. **创建服务器参数**：配置如何启动并连接到 MCP 服务器
2. **建立连接**：使用 `stdio_client` 创建通信通道
3. **初始化会话**：使用通信通道设置 MCP 会话
4. **发现工具**：调用 `list_tools()` 获取所有可用工具
5. **格式化结果**：将工具转换为对我们的代理更有用的格式

我们正在使用异步方法 (`async/await`)，因为 MCP 操作在设计上是非阻塞的。这在主机实现中很重要，因为它允许我们的代理并发处理多个操作，即使在等待工具操作完成时也能保持响应。

让我们测试我们的工具发现功能，以确保其正常工作：


In [3]:
# Test the tool discovery function
tools = await discover_tools()
print(f"Discovered {len(tools)} tools:")
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool['name']}: {tool['description']}")

🔍 DISCOVERY PHASE: Connecting to MCP server...
📡 Initializing MCP connection...
🔎 Discovering available tools...
✅ Successfully discovered 2 tools
Discovered 2 tools:
1. get_crypto_price: 
    Get the current price of a cryptocurrency in a specified currency.
    
    Parameters:
    - crypto_id: The ID of the cryptocurrency (e.g., 'bitcoin', 'ethereum')
    - currency: The currency to display the price in (default: 'usd')
    
    Returns:
    - Current price information as a formatted string
    
2. get_crypto_market_info: 
    Get market information for one or more cryptocurrencies.
    
    Parameters:
    - crypto_ids: Comma-separated list of cryptocurrency IDs (e.g., 'bitcoin,ethereum')
    - currency: The currency to display values in (default: 'usd')
    
    Returns:
    - Market information including price, market cap, volume, and price changes
    


当我们运行此代码时，我们应该看到 MCP 服务器提供的可用工具列表。在这种情况下，我们期望看到我们的加密货币工具。

### 工具执行：实现我们的 MCP 客户端

既然我们的主机可以发现可用工具，我们需要实现可以执行它们的客户端组件。与可能内置此功能的第三方工具不同，我们正在创建自己的客户端，以完全控制和透明地执行 MCP 工具：


In [4]:
async def execute_tool(tool_name: str, arguments: Dict[str, Any]):
    """
    Execute a specific tool provided by the MCP server.
    
    Args:
        tool_name: The name of the tool to execute
        arguments: A dictionary of arguments to pass to the tool
        
    Returns:
        The result from executing the tool
    """
    # ANSI color codes for better log visibility
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RESET = "\033[0m"
    SEP = "-" * 40
    
    server_params = StdioServerParameters(
        command="python",
        args=[mcp_server_path],
    )
    
    print(f"{YELLOW}{SEP}")
    print(f"⚙️ EXECUTION PHASE: Running tool '{tool_name}'")
    print(f"📋 Arguments: {json.dumps(arguments, indent=2)}")
    print(f"{SEP}{RESET}")
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # Call the specific tool with the provided arguments
            print(f"{BLUE}📡 Sending request to MCP server...{RESET}")
            result = await session.call_tool(tool_name, arguments)
            
            print(f"{GREEN}✅ Tool execution complete{RESET}")
            
            # Format result preview for cleaner output
            result_preview = str(result)
            if len(result_preview) > 150:
                result_preview = result_preview[:147] + "..."
                
            print(f"{BLUE}📊 Result: {result_preview}{RESET}")
            print(f"{SEP}")
            
            return result

print("Tool execution function defined")

Tool execution function defined


此函数构成了我们 MCP 客户端的核心：

1. **连接到服务器**：类似于我们的发现功能，它建立到 MCP 服务器的连接
2. **执行工具**：使用提供的参数调用指定的工具
3. **返回结果**：返回工具返回的任何内容

请注意，对于每次工具执行，我们都会创建一个到 MCP 服务器的新连接。虽然这看起来效率低下，但它确保了工具调用之间的清晰分离，并避免了潜在的状态问题。这种无状态方法简化了我们的实现，并使其更加稳健。在生产系统中，您可以通过维护持久连接来优化这一点，但目前的方法对于教育目的来说非常好，因为它清楚地分离了过程中的每个步骤。

既然我们有了发现和执行工具的功能，我们需要将这些功能与可以确定何时以及如何使用它们的 AI 集成。这就是 Claude 发挥作用的地方。

### 将 AI 与我们的 MCP 实现集成

随着主机和客户端组件的到位，我们现在需要将它们与可以对工具使用做出明智决策的 AI 系统集成。这是我们自定义 MCP 主机的“大脑”，它需要：
1. 根据用户输入了解何时需要工具
2. 为任务选择合适的工具
3. 正确格式化参数
4. 处理并解释结果

让我们实现一个编排整个过程的函数：

In [5]:
async def query_claude(prompt: str, tool_info: List[Dict], previous_messages=None):
    """
    Send a query to Claude and process the response.
    
    Args:
        prompt: User's query
        tool_info: Information about available tools
        previous_messages: Previous messages for maintaining context
        
    Returns:
        Claude's response, potentially after executing tools
    """
    # ANSI color codes for better log visibility
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    PURPLE = "\033[95m"
    RESET = "\033[0m"
    SEP = "=" * 40
    
    if previous_messages is None:
        previous_messages = []
    
    print(f"{PURPLE}{SEP}")
    print("🧠 REASONING PHASE: Processing query with Claude")
    print(f"🔤 Query: \"{prompt}\"")
    print(f"{SEP}{RESET}")
    
    # Format tool information for Claude
    tool_descriptions = "\n\n".join([
        f"Tool: {tool['name']}\nDescription: {tool['description']}\nSchema: {json.dumps(tool['schema'], indent=2)}"
        for tool in tool_info
    ])
    
    # Build the system prompt
    system_prompt = f"""You are an AI assistant with access to specialized tools through MCP (Model Context Protocol).
    
Available tools:
{tool_descriptions}

When you need to use a tool, respond with a JSON object in the following format:
{{
    "tool": "tool_name",
    "arguments": {{
        "arg1": "value1",
        "arg2": "value2"
    }}
}}

Do not include any other text when using a tool, just the JSON object.
For regular responses, simply respond normally.
"""
    
    # Filter out system messages from previous messages
    filtered_messages = [msg for msg in previous_messages if msg["role"] != "system"]
    
    # Build the messages for the conversation (WITHOUT system message)
    messages = filtered_messages.copy()
    
    # Add the current user query
    messages.append({"role": "user", "content": prompt})
    
    print(f"{BLUE}📡 Sending request to Claude API...{RESET}")
    
    # Send the request to Claude with system as a top-level parameter
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=4000,
        system=system_prompt,  # System prompt as a separate parameter
        messages=messages      # Only user and assistant messages
    )
    
    # Get Claude's response
    claude_response = response.content[0].text
    print(f"{GREEN}✅ Received response from Claude{RESET}")
    
    # Try to extract and parse JSON from the response
    try:
        # Look for JSON pattern in the response
        import re
        json_match = re.search(r'(\{[\s\S]*\})', claude_response)
        
        if json_match:
            json_str = json_match.group(1)
            print(f"{YELLOW}🔍 Tool usage detected in response{RESET}")
            print(f"{BLUE}📦 Extracted JSON: {json_str}{RESET}")
            
            tool_request = json.loads(json_str)
            
            if "tool" in tool_request and "arguments" in tool_request:
                tool_name = tool_request["tool"]
                arguments = tool_request["arguments"]
                
                print(f"{YELLOW}🔧 Claude wants to use tool: {tool_name}{RESET}")
                
                # Execute the tool using our MCP client
                tool_result = await execute_tool(tool_name, arguments)
                
                # Convert tool result to string if needed
                if not isinstance(tool_result, str):
                    tool_result = str(tool_result)
                
                # Update messages with the tool request and result
                messages.append({"role": "assistant", "content": claude_response})
                messages.append({"role": "user", "content": f"Tool result: {tool_result}"})
                
                print(f"{PURPLE}🔄 Getting Claude's interpretation of the tool result...{RESET}")
                
                # Get Claude's interpretation of the tool result
                final_response = client.messages.create(
                    model="claude-3-5-sonnet-20240620",
                    max_tokens=4000,
                    system=system_prompt,
                    messages=messages
                )
                
                print(f"{GREEN}✅ Final response ready{RESET}")
                print(f"{SEP}")
                
                return final_response.content[0].text, messages
        
    except (json.JSONDecodeError, KeyError, AttributeError) as e:
        print(f"{YELLOW}⚠️ No tool usage detected in response: {str(e)}{RESET}")
    
    print(f"{GREEN}✅ Response ready{RESET}")
    print(f"{SEP}")
    
    return claude_response, messages

print("Claude query function defined")

Claude query function defined


此函数通过复杂的推理和执行流程完成了我们的自定义 MCP 主机实现：

1. **工具描述**：我们以 Claude 可以理解的方式格式化工具信息
2. **系统提示**：我们就何时以及如何使用工具提供说明
3. **响应分析**：我们在 Claude 的响应中查找 JSON 工具请求
4. **工具执行**：如果检测到工具请求，我们使用客户端执行相应的工具
5. **结果处理**：我们将工具结果发送回 Claude 进行解释
6. **对话管理**：我们通过跟踪消息来维护上下文

这产生了强大的协同效应：Claude 提供推理和沟通技巧，而我们的 MCP 工具提供专门的能力和实时数据访问。

让我们用一个关于比特币价格的简单查询来测试我们的代理：

In [ ]:
# Run a single query using the tools from your MCP server
query = "What is the current price of Bitcoin?"
print(f"Sending query: {query}")

response, messages = await query_claude(query, tools)
print(f"\nAssistant's response:\n{response}")


Sending query: What is the current price of Bitcoin?
🧠 REASONING PHASE: Processing query with Claude
🔤 Query: "What is the current price of Bitcoin?"
📡 Sending request to Claude API...
✅ Received response from Claude
🔍 Tool usage detected in response
📦 Extracted JSON: {
    "tool": "get_crypto_price",
    "arguments": {
        "crypto_id": "bitcoin"
    }
}
🔧 Claude wants to use tool: get_crypto_price
----------------------------------------
⚙️ EXECUTION PHASE: Running tool 'get_crypto_price'
📋 Arguments: {
  "crypto_id": "bitcoin"
}
----------------------------------------
📡 Sending request to MCP server...
✅ Tool execution complete
📊 Result: meta=None content=[TextContent(type='text', text='The current price of bitcoin is 83667 USD', annotations=None)] isError=False
----------------------------------------
🔄 Getting Claude's interpretation of the tool result...
✅ Final response ready

Assistant's response:
Based on the tool result, I can provide you with the current price of Bitcoin

当我们运行此查询时，我们完整的 MCP 实现遵循此流程：
1. Claude（通过我们的主机）将其识别为关于比特币价格的请求
2. 我们的 AI 决定使用 `get_crypto_price` 工具
3. 它正确格式化参数（使用 "bitcoin" 作为 crypto_id）
4. 我们的客户端连接到服务器并执行工具，返回当前的比特币价格
5. Claude 使用附加上下文以自然语言解释结果

这展示了我们代理的全部能力：理解用户的意图，选择合适的工具，正确执行它，并提供有用的、上下文丰富的响应。

### 通过我们的客户端直接执行工具

虽然我们的集成 MCP 主机通常根据用户的查询决定使用哪些工具，但有时我们可能希望直接使用我们的客户端来执行特定的工具。这对于测试我们的客户端实现或演示特定的工具功能很有用。让我们创建一个简单的例子：

In [ ]:
try:
    # Get the first tool name from your discovered tools
    if tools:
        first_tool = tools[0]
        tool_name = first_tool["name"]
        
        # Use the correct parameter name for get_crypto_price
        arguments = {"crypto_id": "bitcoin"}
        
        print(f"Executing tool '{tool_name}' with arguments: {arguments}")
        result = await execute_tool(tool_name, arguments)
        print(f"Tool result: {result}")
    else:
        print("No tools discovered to test")
except Exception as e:
    print(f"Error executing tool: {str(e)}")

Executing tool 'get_crypto_price' with arguments: {'crypto_id': 'bitcoin'}
----------------------------------------
⚙️ EXECUTION PHASE: Running tool 'get_crypto_price'
📋 Arguments: {
  "crypto_id": "bitcoin"
}
----------------------------------------
📡 Sending request to MCP server...
✅ Tool execution complete
📊 Result: meta=None content=[TextContent(type='text', text='The current price of bitcoin is 83670 USD', annotations=None)] isError=False
----------------------------------------
Tool result: meta=None content=[TextContent(type='text', text='The current price of bitcoin is 83670 USD', annotations=None)] isError=False


这种直接执行方法对于以下情况很有用：
- 隔离测试我们的客户端实现
- 调试工具功能
- 构建工具执行已预先确定的专用工作流
- 在将 MCP 客户端与 AI 集成之前验证其是否正常工作

现在，让我们创建一个使用我们完整 MCP 主机实现的交互式聊天界面：

### 构建交互式 MCP 主机界面

对于完整的 MCP 主机实现，我们需要一个在多轮对话中保持上下文的用户界面。这允许我们的主机记住以前的交互并在随后的交流中建立它们，就像 Claude Desktop 这样的专业 MCP 主机一样。让我们实现一个简单的聊天会话功能：

In [ ]:
async def chat_session():
    """
    Run an interactive chat session with the AI agent.
    """
    # ANSI color codes for better log visibility
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    CYAN = "\033[96m"
    BOLD = "\033[1m"
    RESET = "\033[0m"
    SEP = "=" * 50
    
    print(f"{CYAN}{BOLD}{SEP}")
    print("🤖 INITIALIZING MCP AGENT")
    print(f"{SEP}{RESET}")
    
    # Make sure 'tools' is defined from a previous cell, or discover them again
    try:
        # Check if tools is defined and not empty
        if 'tools' not in globals() or not tools:
            print(f"{BLUE}🔍 No tools found, discovering available tools...{RESET}")
            tools_local = await discover_tools()
        else:
            tools_local = tools
            
        print(f"{GREEN}✅ Agent ready with {len(tools_local)} tools:{RESET}")
        
        # Print the available tools for reference
        for i, tool in enumerate(tools_local, 1):
            print(f"{YELLOW}  {i}. {tool['name']}{RESET}")
            print(f"     {tool['description'].strip()}")
        
        # Start the chat session
        print(f"\n{CYAN}{BOLD}{SEP}")
        print(f"💬 INTERACTIVE CHAT SESSION")
        print(f"{SEP}")
        print(f"Type 'exit' or 'quit' to end the session{RESET}")
        
        messages = []
        
        while True:
            # Get user input
            user_input = input(f"\n{BOLD}You:{RESET} ")
            
            # Check if user wants to exit
            if user_input.lower() in ['exit', 'quit']:
                print(f"\n{GREEN}Ending chat session. Goodbye!{RESET}")
                break
            
            # Process the query with Claude
            print(f"\n{BLUE}Processing...{RESET}")
            response, messages = await query_claude(user_input, tools_local, messages)
            
            # Display Claude's response
            print(f"\n{BOLD}Assistant:{RESET} {response}")
            
    except Exception as e:
        print(f"\n{YELLOW}⚠️ An error occurred: {str(e)}{RESET}")

print("Chat session function defined. Run 'await chat_session()' in the next cell to start chatting.")

Chat session function defined. Run 'await chat_session()' in the next cell to start chatting.


我们的 MCP 主机界面：

1. **初始化工具**：我们的主机在启动时发现可用工具
2. **创建会话循环**：持续提示用户输入
3. **维护上下文**：将以前的消息传递给每个查询，维护有状态的对话
4. **处理优雅退出**：允许用户优雅地结束会话

这创造了一种自然的、对话式的体验，代理可以记住以前的交互。例如，如果用户询问比特币，然后接着问“以太坊怎么样？”，代理就会理解上下文。

现在，让我们运行我们的聊天会话，看看完整的代理在行动：

您可以尝试我们在 Claude Desktop 中提出的问题：What's the market data for Dogecoin and Solana?

In [ ]:
# Run the chat session
await chat_session()

🤖 INITIALIZING MCP AGENT
✅ Agent ready with 2 tools:
  1. get_crypto_price
     Get the current price of a cryptocurrency in a specified currency.
    
    Parameters:
    - crypto_id: The ID of the cryptocurrency (e.g., 'bitcoin', 'ethereum')
    - currency: The currency to display the price in (default: 'usd')
    
    Returns:
    - Current price information as a formatted string
  2. get_crypto_market_info
     Get market information for one or more cryptocurrencies.
    
    Parameters:
    - crypto_ids: Comma-separated list of cryptocurrency IDs (e.g., 'bitcoin,ethereum')
    - currency: The currency to display values in (default: 'usd')
    
    Returns:
    - Market information including price, market cap, volume, and price changes

💬 INTERACTIVE CHAT SESSION
Type 'exit' or 'quit' to end the session

Processing...
🧠 REASONING PHASE: Processing query with Claude
🔤 Query: "What's the market data for Dogecoin and Solana?"
📡 Sending request to Claude API...
✅ Received response from

有了这最后一块，我们创建了一个完整的自定义 MCP 主机实现，它可以：
1. 作为客户端连接到 MCP 服务器
2. 发现可用工具
3. 智能地选择和使用这些工具来回答用户查询
4. 在对话中维护上下文

这展示了实现我们自己的 MCP 主机和客户端的力量 - 我们完全控制 AI 如何与工具交互，同时保持 MCP 协议标准化的所有好处。

## 结论：

模型上下文协议代表了一种将 AI 模型与外部资源集成的变革性方法，解决了 AI 应用程序开发中的关键挑战：

### 协议优势
- **标准化集成**：消除复杂的自定义 API 连接
- **动态工具发现**：使 AI 能够无缝地发现和使用工具
- **灵活通信**：支持实时、双向交互

### 技术亮点
我们的实现演示了：
- 使用专用工具构建 MCP 服务器
- 创建一个可以动态发现和执行工具的主机
- 将 AI 模型与外部资源集成

### 关键引用

- [Python MCP SDK](https://github.com/modelcontextprotocol/python-sdk)
- [MCP 快速入门指南](https://modelcontextprotocol.io/quickstart/user)
- [CoinGecko API 文档](https://www.coingecko.com/en/api)